# RNA velocity analysis

We perform the analysis on the batches SIGAH5 (IVF) and SIGAH12 (NT)

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import scvelo as scv

In [ ]:
import cellrank as cr
from cellrank.tl.kernels import VelocityKernel
from cellrank.tl.kernels import ConnectivityKernel
from cellrank.tl.estimators import GPCCA

In [ ]:
print(scv.__version__)
print(cr.__version__)
print(sc.__version__)
print(ad.__version__)

In [ ]:
scv.logging.print_version()

In [ ]:
import matplotlib

matplotlib.__version__

In [ ]:
# Load the files with metadata informations
rnavelo_folder='./data_utils/'

# Load the first 2 UMAP components (original UMAP used for clustering)
umap_tot=pd.read_csv(rnavelo_folder+'full_umap.csv',index_col=0)

umap_tot.index=umap_tot.index.str.replace('.1', '')

cell_names_SIGAH5=np.loadtxt(rnavelo_folder+'SIGAH5_cell_names.txt',dtype=str)
cell_names_SIGAH12=np.loadtxt(rnavelo_folder+'SIGAH12_cell_names.txt',dtype=str)

cell_names_SIGAH5=[s.replace('.1', '') for s in list(cell_names_SIGAH5)]
cell_names_SIGAH12=[s.replace('.1', '') for s in list(cell_names_SIGAH12)]

my_color_palette=np.loadtxt(rnavelo_folder+'cluster_colors.txt',dtype=str,comments='/')

In [ ]:
print(len(cell_names_SIGAH5),len(set(cell_names_SIGAH5)))
print(len(cell_names_SIGAH12),len(set(cell_names_SIGAH12)))

In [ ]:
# Load Chris' loom files
loom_folder='./loom_files/'

adata_SIGAH5=scv.read(loom_folder+'SIGAH5_IVF.loom', sparse=True,cache=True)
adata_SIGAH12=scv.read(loom_folder+'SIGAH12_NT.loom', sparse=True,cache=True)

adata_SIGAH5.var_names_make_unique()
adata_SIGAH12.var_names_make_unique()

adata_SIGAH5.obs.index=adata_SIGAH5.obs.index.str.replace('SIGAH5_91_Chm3_FC2:', '')
adata_SIGAH5.obs.index=adata_SIGAH5.obs.index.str.replace('x', '')

adata_SIGAH12.obs.index=adata_SIGAH12.obs.index.str.replace('SIGAH12_91_Chm3_FC2:', '')
adata_SIGAH12.obs.index=adata_SIGAH12.obs.index.str.replace('x', '')

# We eliminated cells from cluster 10 and outlier cells from cluster 6 from our dataset,
# so we eliminate them accordingly also from the matrices obtained from the loom files
adata_SIGAH5=adata_SIGAH5[cell_names_SIGAH5,:]
adata_SIGAH12=adata_SIGAH12[cell_names_SIGAH12,:]

# Assign umap coordinates and cluster labels to each batch

# Original UMAP
umap_SIGAH5_tot=umap_tot.loc[cell_names_SIGAH5,:]
umap_SIGAH12_tot=umap_tot.loc[cell_names_SIGAH12,:]

cluster_labels_SIGAH5=cluster_labels_exp2.loc[cell_names_SIGAH5,:]
cluster_labels_SIGAH12=cluster_labels_exp2.loc[cell_names_SIGAH12,:]

# Check the order of the cells, assign UMAP coordinates and cluster labels

# Original UMAP
umap_SIGAH5_tot=umap_SIGAH5_tot.reindex(adata_SIGAH5.obs_names)
umap_SIGAH12_tot=umap_SIGAH12_tot.reindex(adata_SIGAH12.obs_names)

cluster_labels_SIGAH5=cluster_labels_SIGAH5.reindex(adata_SIGAH5.obs_names)
cluster_labels_SIGAH12=cluster_labels_SIGAH12.reindex(adata_SIGAH12.obs_names)

In [ ]:
# Check order of the cells
print('Original UMAP coords')
print(list(adata_SIGAH5.obs_names) == list(umap_SIGAH5_tot.index))
print(list(adata_SIGAH12.obs_names) == list(umap_SIGAH12_tot.index))

print('Cluster labels')
print(list(adata_SIGAH5.obs_names) == list(cluster_labels_SIGAH5.index))
print(list(adata_SIGAH12.obs_names) == list(cluster_labels_SIGAH12.index))

In [ ]:
adata_SIGAH5.obs['seurat_clusters']=list(cluster_labels_SIGAH5['seurat_clusters'])
adata_SIGAH12.obs['seurat_clusters']=list(cluster_labels_SIGAH12['seurat_clusters'])

# Original UMAP stored in X_umap
adata_SIGAH5.obsm['X_umap']=np.array(umap_SIGAH5_tot[['UMAP_1','UMAP_2']])
adata_SIGAH12.obsm['X_umap']=np.array(umap_SIGAH12_tot[['UMAP_1','UMAP_2']])

adata_SIGAH5.obs['seurat_clusters']=adata_SIGAH5.obs['seurat_clusters'].astype('category')
adata_SIGAH12.obs['seurat_clusters']=adata_SIGAH12.obs['seurat_clusters'].astype('category')

adata_SIGAH5.uns['seurat_clusters_colors']=my_color_palette
adata_SIGAH12.uns['seurat_clusters_colors']=my_color_palette

In [ ]:
def RNAVeloWrap(adata,batch,label,cond,cl_lab):
    scv.utils.show_proportions(adata)
    scv.pp.filter_genes(adata,min_cells=10)
    scv.pp.filter_and_normalize(adata, min_counts=20, min_counts_u=10, n_top_genes=2000)
    adata.raw=adata
    scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
    scv.tl.recover_dynamics(adata)
    scv.tl.velocity(adata, mode='dynamical')
    scv.tl.velocity_graph(adata)
    scv.tl.velocity_embedding(adata, basis=my_basis) 
    scv.tl.paga(adata, groups=cl_lab)
    df = scv.get_df(adata, 'paga/transitions_confidence', precision=2).T
    print('velo genes',adata.var['velocity_genes'].sum())

In [ ]:
# Run RNA velocity analysis
RNAVeloWrap(adata_SIGAH5,batch='SIGAH5',label='total',cond='IVF',cl_lab='seurat_clusters')
RNAVeloWrap(adata_SIGAH12,batch='SIGAH12',label='total',cond='NT',cl_lab='seurat_clusters')

# Save the results of RNA velocity analysis

In [ ]:
velo_folder='./RNA_velo_processed_data/'
adata_SIGAH5.write_h5ad(velo_folder+'data_SIGAH5.h5ad')
adata_SIGAH12.write_h5ad(velo_folder+'data_SIGAH12.h5ad')